In [ ]:
!pip install -qq unsloth
# !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
# !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
# !pip install --no-deps unsloth

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 2.7.0 which is incompatible.


In [16]:
import torch
from datasets import load_dataset
from unsloth import FastModel
from transformers import TextStreamer

In [2]:
reasoning_start = "<start_working_out>"
reasoning_end = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

SYSTEM_PROMPT = f"""You are a mini-Sudoku solving assistant.
You will be given a 4x4 Sudoku puzzle where some cells are filled and others are empty (shown as spaces).
The goal is to fill each empty cell with a number from 1 to 4 such that:
- Each row contains all numbers from 1 to 4 exactly once
- Each column contains all numbers from 1 to 4 exactly once
- Each 2x2 sub-grid contains all numbers from 1 to 4 exactly once

Think through the solution step by step.
Place your reasoning between {reasoning_start} and {reasoning_end}.
Then, provide your complete 4x4 solution grid between {solution_start} and {solution_end}

The solution should be formatted as a 4x4 grid with spaces between numbers and newlines between rows.
For example:
{solution_start}
1 2 3 4
3 4 1 2
2 1 4 3
4 3 2 1
{solution_end}
"""

In [3]:
train_dataset = load_dataset("asadshahab/mini-sudoku", split="train")
val_dataset = load_dataset("asadshahab/mini-sudoku", split="validation")

In [4]:
def show(s):
  for line in s.split("\n"):
    print(line)

In [5]:
show(train_dataset[0]["question"])

4 _ _ _
_ 3 _ _
_ 1 3 _
_ _ _ _


In [6]:
show(train_dataset[0]["answer"])

4 2 1 3
1 3 4 2
2 1 3 4
3 4 2 1


In [7]:
def parse_sudoku_question(q):
  return q.replace("_", " ")

def preprocess_dataset(example):
  return {
      "prompt": [
          {"role": "system", "content": SYSTEM_PROMPT},
          {"role": "user", "content": parse_sudoku_question(example["question"])}
      ],
      "answer": example["answer"],
  }

In [8]:
processed_train_dataset = train_dataset.map(preprocess_dataset)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

In [9]:
processed_train_dataset[0]

{'question': '4 _ _ _\n_ 3 _ _\n_ 1 3 _\n_ _ _ _',
 'answer': '4 2 1 3\n1 3 4 2\n2 1 3 4\n3 4 2 1',
 'prompt': [{'content': 'You are a mini-Sudoku solving assistant.\nYou will be given a 4x4 Sudoku puzzle where some cells are filled and others are empty (shown as spaces).\nThe goal is to fill each empty cell with a number from 1 to 4 such that:\n- Each row contains all numbers from 1 to 4 exactly once\n- Each column contains all numbers from 1 to 4 exactly once\n- Each 2x2 sub-grid contains all numbers from 1 to 4 exactly once\n\nThink through the solution step by step.\nPlace your reasoning between <start_working_out> and <end_working_out>.\nThen, provide your complete 4x4 solution grid between <SOLUTION> and </SOLUTION>\n\nThe solution should be formatted as a 4x4 grid with spaces between numbers and newlines between rows.\nFor example:\n<SOLUTION>\n1 2 3 4\n3 4 1 2\n2 1 4 3\n4 3 2 1\n</SOLUTION>\n',
   'role': 'system'},
  {'content': '4      \n  3    \n  1 3  \n       ', 'role': 'u

In [10]:
max_seq_length = 1024
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [11]:
model, tokenizer = FastModel.from_pretrained(
    model_name="unsloth/gemma-3-4b-it",
    max_seq_length=max_seq_length,
    dtype=torch.bfloat16,
    load_in_4bit=False,
    load_in_8bit=False,
    full_finetuning=True,
)

==((====))==  Unsloth 2025.6.5: Fast Gemma3 patching. Transformers: 4.52.4.
   \\   /|    NVIDIA A40. Num GPUs = 1. Max memory: 44.422 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 8.6. CUDA Toolkit: 12.6. Triton: 3.3.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using bfloat16 full finetuning which cuts memory usage by 50%.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [13]:
# inference

parsed_puzzle = processed_train_dataset[0]["question"]
show(parsed_puzzle)

4 _ _ _
_ 3 _ _
_ 1 3 _
_ _ _ _


In [14]:
messages = [
  {"role": "system", "content": SYSTEM_PROMPT},
  {"role": "user", "content": parsed_puzzle},
]
text = tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False)

In [18]:
outputs = model.generate(
  **tokenizer(text, return_tensors="pt").to(device),
  max_new_tokens=max_seq_length,
  # temperature=0.0,
  top_p=0.95,
  top_k=64,
  streamer=TextStreamer(tokenizer, skip_prompt=True),
)

<start_working_out>
Let's analyze the given 4x4 Sudoku puzzle:

4 _ _ _
_ 3 _ _
_ 1 3 _
_ _ _ _

We can start by looking for cells where only one number is possible.

In the first row, we have 4, so the remaining numbers are 1, 2, and 3.
In the second row, we have 3, so the remaining numbers are 1, 2, and 4.
In the third row, we have 1 and 3, so the remaining numbers are 2 and 4.
In the fourth row, we have no numbers yet.

Let's consider the first column. We have 4 and no other numbers. Thus the first column must have 1, 2, and 3.

Let's analyze the top-left 2x2 block. We have 4 and 3.  The missing numbers are 1 and 2.
In the second row, the first block already has 3, so the first cell in the second row must be either 1 or 2.
In the third row, the first cell must be either 2 or 4.

Let's try placing 2 in the second row first cell.
4 _ _ _
_ 3 _ _
_ 1 3 _
_ _ _ _

Now, in the first column, we have 4, so the other numbers must be 1, 2, and 3. In the second row, we have 3, so the other nu

In [20]:
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
# print(generated_text)

In [30]:
# extracting answer
import re
match_format = re.compile(rf"{solution_start}(.+?){solution_end}", flags=re.MULTILINE | re.DOTALL)
match = match_format.search(generated_text)
match.group(1).strip() ## there's multiple tags and finds the first match.. FIX!

'and'

In [32]:
from trl import GRPOConfig, GRPOTrainer
use_wandb = False

max_prompt_length = 256
training_args = GRPOConfig(
  learning_rate=5e-6,
  adam_beta1=0.9,
  adam_beta2=0.99,
  weight_decay=0.1,
  warmup_ratio=0.1,
  lr_scheduler_type="cosine",
  optim="adamw_torch_fused",
  logging_steps=10,
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  num_generations=6,
  max_prompt_length=max_prompt_length,
  max_completion_length=max_seq_length - max_prompt_length,
  num_train_epochs=4,
  save_steps=100,
  report_to="wandb" if use_wandb else "none",
  output_dir="outputs",
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 4 to the `num_generations` of 6


In [33]:
def match_format_exactly(completions, **kwargs):
    """Check if output follows the required format. Max: 2.0"""
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Clean up any end_of_turn markers
        response = response.replace("<end_of_turn>", "")
        # Try to find solution tags
        match_format = re.compile(
            rf"^[\s]{{0,}}"
            rf"{reasoning_start}.+?{reasoning_end}.*?"
            rf"{solution_start}(.+?){solution_end}"
            rf"[\s]*"
            rf"(?:<end_of_turn>)?"
            rf"[\s]*$",
            flags=re.MULTILINE | re.DOTALL
        )
        if match_format.search(response) is not None:
            score += 2.0
        scores.append(score)
    return scores


def correctness_reward_func(prompts, completions, answer, **kwargs):
    """Reward function that checks if the Sudoku solution is correct. Max: 5.0"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    
    for response, correct_answer in zip(responses, answer):
        solution_text = extract_solution(response)
        if solution_text is None:
            rewards.append(0.0)
            continue
            
        predicted_grid = extract_grid_from_answer(solution_text)
        correct_grid = extract_grid_from_answer(correct_answer)
        
        if predicted_grid is None or correct_grid is None:
            rewards.append(0.0)
            continue
            
        if predicted_grid == correct_grid and is_valid_sudoku_solution(predicted_grid):
            rewards.append(5.0)
        else:
            rewards.append(0.0)
            
    return rewards

def int_reward_func(completions, **kwargs):
    """Reward function that checks if all numbers in the solution are 1-4. Max: 0.5"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    
    for response in responses:
        solution_text = extract_solution(response)
        if solution_text is None:
            rewards.append(0.0)
            continue
            
        grid = extract_grid_from_answer(solution_text)
        if grid is None:
            rewards.append(0.0)
            continue
            
        try:
            if all(all(num in [1, 2, 3, 4] for num in row) for row in grid):
                rewards.append(0.5)
            else:
                rewards.append(0.0)
        except:
            rewards.append(0.0)
            
    return rewards

def grid_format_reward_func(completions, **kwargs):
    """Reward function for checking the format of the output. Max: 1.0"""
    responses = [completion[0]['content'] for completion in completions]
    rewards = []
    
    for response in responses:
        solution_text = extract_solution(response)
        if solution_text is None:
            rewards.append(0.0)
            continue
            
        # Check if the solution has a grid-like structure
        lines = solution_text.strip().split('\n')
        valid_lines = 0
        
        for line in lines:
            # Check if line contains 4 numbers (1-4) with possible spaces
            if re.match(r'^\s*[1-4](\s+[1-4]){3}\s*$', line):
                valid_lines += 1
                
        # Give reward based on how many valid lines we found
        if valid_lines == 4:
            rewards.append(1.0)
        elif valid_lines > 0:
            rewards.append(valid_lines / 8.0)  # Partial reward
        else:
            rewards.append(0.0)
            
    return rewards

In [34]:
trainer = GRPOTrainer(
  model=model,
  tokenizer=tokenizer,
  reward_funcs=[
    match_format_exactly,  
    correctness_reward_func,
    int_reward_func,
    grid_format_reward_func,
  ],
  args=training_args,
  train_dataset=processed_train_dataset,
)

print("\nStarting training...")
trainer.train()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


Starting training...


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,800 | Num Epochs = 4 | Total steps = 1,800
O^O/ \_/ \    Batch size per device = 6 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (6 x 4 x 1) = 24
 "-____-"     Trainable parameters = 4,300,079,472/4,300,079,472 (100.00% trained)
`generation_config` default values have been modified to match model-specific defaults: {'top_p': 0.95}. If this is not desired, please set these values explicitly.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


OutOfMemoryError: CUDA out of memory. Tried to allocate 9.25 GiB. GPU 0 has a total capacity of 44.42 GiB of which 7.26 GiB is free. Process 505705 has 37.16 GiB memory in use. Of the allocated memory 36.28 GiB is allocated by PyTorch, and 556.58 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)